In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["PREFECT__FLOWS__CHECKPOINTING"] = "True"

import fsspec
import intake
import pandas as pd
import xarray as xr
import xesmf as xe
import zarr
from prefect import Flow, Parameter, task
import re
import numpy as np
from xarray_schema import DataArraySchema, DatasetSchema

from cmip6_downscaling.workflows.utils import rechunk_zarr_array, calc_auspicious_chunks_dict, delete_chunks_encoding, regrid_dataset, load_paths
import cmip6_downscaling
import numpy as np
from xarray_schema import DataArraySchema, DatasetSchema
connection_string = "DefaultEndpointsProtocol=https;AccountName=cmip6downscaling;AccountKey=Z86Ca7uttNw4smgPikZFgsB1Lgf8GFpL+lMiViWRaovBSsOWK5JFXAvlk5rO18R/61wqSfUDolvd8olXLt0law==;EndpointSuffix=core.windows.net"

os.environ['AZURE_STORAGE_CONNECTION_STRING'] = connection_string
from cmip6_downscaling.data.observations import open_era5
from cmip6_downscaling.data.cmip import load_cmip, gcm_munge, convert_to_360
# from cmip6_downscaling.methods.bcsd import preprocess_bcsd, prep_bcsd_inputs, fit_and_predict, postprocess_bcsd, make_flow_paths
from dask.distributed import Client, LocalCluster
import warnings
import rechunker
warnings.filterwarnings("ignore")
import pytest
from cmip6_downscaling import analysis
from cmip6_downscaling.analysis import load
from cmip6_downscaling.analysis.analysis import qaqc_checks
from cmip6_downscaling.analysis.qaqc import make_qaqc_ds
import matplotlib.pyplot as plt

In [2]:
# cluster = LocalCluster()
# client = Client(cluster)
# cluster.scale(4)


In [91]:
# Pass as a parameter to be used by papermill: the runid which will be used to create the template for the raw and summary files
run_id = 'MIROC6_ssp370_1991_1995_2071_2075_tasmax_21_day_3_year_rolling'

In [92]:
# store = fsspec.get_mapper(f'az://cmip6downscaling/flow-outputs/intermediate/epoch_replaced_gcm/{run_id}.zarr')

In [93]:
ds = xr.open_zarr(f'az://flow-outputs/intermediate/epoch_replaced_gcm/{run_id}.zarr')

# QAQC Routines
Search for nans and aphysical quantitities

In [12]:
annual_qaqc_ts, qaqc_maps = qaqc_checks(ds.isel(time=slice(0,2)))

What years, if any, include QAQC issues?

In [34]:
if annual_qaqc_ts.sum().values.any():
    print(annual_qaqc_ts)

Where, if anywhere, do those QAQC issues happen?

In [46]:
if qaqc_maps[parameters['VARIABLE']].sum().values.any():
    qaqc_maps[parameters['VARIABLE']].plot(col_wrap=1, col='qaqc_check')

In [ ]:
method = np.max

In [ ]:
summarizing_method = {'tmax': np.max,
                      'tmin': np.min,
                     'wind': np.max,}
time_aggregations = ['month', 'year']

# Metrics

In [ ]:
# historical_ds = load_obs(start_year, end_year)
# def historical_skill_timeseries()

# def historical_skill_maps():
# xskillscore calculated along time dimension at different aggregations (

# def compare_with_obs_historical(ds, historical_ds):
#xskillscore